In [1]:
from lstm_model import build_open_loop_lstm, load_open_loop_lstm
from data_processing import create_training_split, df_training_split, train_valid_test_split, df_train_valid_test_split, create_df_3d, create_window_closed_loop, add_new_pred, compute_lyapunov_time_arr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow_datasets as tfds
import tensorflow as tf

import importlib
import feedback_tf_example
importlib.reload(feedback_tf_example)

<module 'feedback_tf_example' from '/Users/eo821/Documents/PhD_research/Lorenz_LSTM/feedback_tf_example.py'>

In [2]:
mydf = np.genfromtxt('CSV/13333_lorenz_normalized.csv', delimiter=',')
time = mydf[0, :]
mydf = mydf[1:, :]
print("Shape of discrete time step array ", time.shape)
print("Shape of solution array: ", mydf.shape)
df_train, df_valid, df_test = df_train_valid_test_split(mydf)
time_train, time_valid, time_test = train_valid_test_split(time)
x_train, x_valid, x_test = train_valid_test_split(mydf[0, :])
y_train, y_valid, y_test = train_valid_test_split(mydf[1, :])
z_train, z_valid, z_test = train_valid_test_split(mydf[2, :])
window_size = 50
batch_size = 32
shuffle_buffer_size = df_train.shape[0]
train_dataset = create_df_3d(df_train.transpose(
), window_size, batch_size, shuffle_buffer_size)
valid_dataset = create_df_3d(df_valid.transpose(), window_size, batch_size, 1)
test_dataset = create_df_3d(df_test.transpose(), window_size, batch_size, 1)

Shape of discrete time step array  (13333,)
Shape of solution array:  (3, 13333)


In [10]:
# only take first element of dataset
for data, labels in train_dataset.take(1):
    numpy_data = data.numpy()
    numpy_labels = labels.numpy()

2022-02-04 11:49:05.439537: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [11]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss', patience=5)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)
history = model_oloop.fit(train_dataset, epochs=100, batch_size=batch_size,
                          validation_data=valid_dataset, callbacks=[tensorboard_callback])  # , early_stop_callback])

NameError: name 'datetime' is not defined

In [46]:
def custom_loss(y_true, y_pred, washout=0):
    mse = tf.keras.losses.MeanSquaredError()
    # (batchsize, dimensions)
    loss = mse(y_true[washout:, :], y_pred[washout:, :])
    return loss

In [6]:
importlib.reload(feedback_tf_example)
feedback_model = feedback_tf_example.FeedBack(units=8)

In [7]:
feedback_model.model_compile()

In [8]:
a, b = feedback_model.warmup(list(train_dataset)[0][0])

2022-02-04 11:49:41.310506: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [1]:
feedback_model.fit(train_dataset, epochs=100, batch_size=32,
                   validation_data=valid_dataset, verbose=2)

NameError: name 'feedback_model' is not defined

In [ ]:
f